# Text-to-Image Generation with Laxhar/noobai-XL-1.1

This notebook allows you to generate images from text prompts using the `Laxhar/noobai-XL-1.1` model. You can customize sampler methods, sampling steps, seeds, CFG scale, apply high-resolution fixes, and adjust image sizes.

In [1]:
# Install necessary libraries
!pip install transformers diffusers accelerate
!pip install --upgrade diffusers

In [2]:
# Import libraries
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from transformers import AutoTokenizer
import IPython.display as display
from PIL import Image

In [3]:
# Load tokenizer and modify character limits
tokenizer = AutoTokenizer.from_pretrained("Laxhar/noobai-XL-1.1")
tokenizer.model_max_length = 1024  # Remove or set to a higher limit
tokenizer_2 = AutoTokenizer.from_pretrained("Laxhar/noobai-XL-1.1", subfolder="tokenizer_2")
tokenizer_2.model_max_length = 1024  # Remove or set to a higher limit

In [4]:
# Define parameters
prompt = "A serene landscape with mountains and a river at sunset"
sampler = "Euler"  # Options: Euler, DDIM, etc.
sampling_steps = 50
seed = 42
cfg_scale = 7.5
image_size = (512, 512)
hires_fix = True
hires_scale = 2.0
hires_steps = 20

In [5]:
# Set random seed for reproducibility
torch.manual_seed(seed)

In [6]:
# Choose sampler
if sampler == "Euler":
    scheduler = EulerDiscreteScheduler.from_config("Laxhar/noobai-XL-1.1")
elif sampler == "DDIM":
    from diffusers import DDIMScheduler
    scheduler = DDIMScheduler.from_config("Laxhar/noobai-XL-1.1")
else:
    raise ValueError("Unsupported sampler method.")

In [7]:
# Load the pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
pipeline = StableDiffusionPipeline.from_pretrained(
    "Laxhar/noobai-XL-1.1",
    scheduler=scheduler,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)

# Enable memory-efficient attention if possible
pipeline.enable_attention_slicing()


In [8]:
# Generate image
with torch.autocast(device):
    image = pipeline(
        prompt,
        num_inference_steps=sampling_steps,
        guidance_scale=cfg_scale,
        height=image_size[0],
        width=image_size[1]
    ).images[0]

display.display(image)

In [9]:
# Apply high-resolution fix if enabled
if hires_fix:
    upscaled_image = image.resize((int(image_size[0]*hires_scale), int(image_size[1]*hires_scale)), Image.LANCZOS)
    with torch.autocast(device):
        upscaled_image = pipeline(
            prompt,
            num_inference_steps=hires_steps,
            guidance_scale=cfg_scale,
            height=upscaled_image.height,
            width=upscaled_image.width,
            init_image=upscaled_image,
            strength=0.8
        ).images[0]
    display.display(upscaled_image)

## Customize Parameters

You can modify the parameters in the **Define parameters** section to customize your image generation.